In [43]:
import torch
import torch.nn as nn

class IoULoss(nn.Module) :
    def __init__(self, method = 'IoU') :
        super().__init__()
        self.method = method

    def forward(self, inp, target) :
        '''
        input : (B, # of bboxes, 6)
        '''
        inp_w = inp[..., 2:3]
        inp_h = inp[..., 3:4]
        target_w = target[..., 2:3]
        target_h = target[..., 3:4]

        inp_area = inp_w * inp_h
        target_area = target_w * target_h

        inp_xmin = inp[..., 0:1] - inp_w / 2
        inp_ymin = inp[..., 1:2] - inp_h / 2
        inp_xmax = inp[..., 0:1] + inp_w / 2
        inp_ymax = inp[..., 1:2] + inp_w / 2

        target_xmin = target[..., 0:1] - target_w / 2
        target_ymin = target[..., 1:2] - target_h / 2
        target_xmax = target[..., 0:1] + target_w / 2
        target_ymax = target[..., 1:2] + target_w / 2
        
        inp_topleft = torch.cat([inp_xmin, inp_ymin], axis = -1)
        target_topleft = torch.cat([target_xmin, target_ymin], axis = -1)

        inp_bottomright = torch.cat([inp_xmax, inp_ymax], axis = -1)
        target_bottomright = torch.cat([target_xmax, target_ymax], axis = -1)

        intersection_top_left = torch.max(inp_topleft, target_topleft)
        intersection_bottom_right = torch.min(inp_bottomright, target_bottomright)



        area_inter = torch.prod(
            torch.clip(intersection_bottom_right - intersection_top_left, min = 0 , max = None), -1).unsqueeze(-1)

        iou = area_inter / (inp_area + target_area - area_inter + 1e-9)

        # GIoU : IoU - |C \ (A U B)| over C. C는 bbox와 GT를 모두 포함하는 최소 크기의 박스.
        C_top_left = torch.min(inp_topleft, target_topleft)
        C_bottom_right = torch.max(inp_bottomright, target_bottomright)
        C_area = torch.prod(C_bottom_right - C_top_left, -1).unsqueeze(-1)

        # DIoU : 중심좌표 반영. 1 - IoU + euclidean(pred_center, gt_center) / diagonal length of C. C는 bbox와 GT를 모두 포함하는 최소 크기의 박스.
        euclidean = torch.sqrt(torch.sum((inp[..., 0:2] - target[..., 0:2]) ** 2, dim = -1)).unsqueeze(-1)
        diagonal_length_C = torch.sqrt(torch.sum((C_bottom_right - C_top_left) ** 2, dim = -1)).unsqueeze(-1)

        if self.method == 'IoU' : 
            return 1 - iou
        elif self.method == 'GIoU' :
            return 1 - (iou - (C_area - (inp_area + target_area - area_inter)) / C_area)
        elif self.method == 'DIoU' :
            return 1 - iou + (euclidean / diagonal_length_C)

In [44]:
# test case 1. IoU 0.36
# test case 2. IoU 1.
# test case 3. IoU 0
# test case 4. IoU 0
pred = torch.tensor([[0.5, 0.5, 0.5, 0.5],
                     [0.5, 0.5, 0.3, 0.3],
                     [0.1, 0.3, 0.2, 0.2],
                     [0.1, 0.3, 0.2, 0.2]
                ]).unsqueeze(0) # for batch

gt = torch.tensor([[0.7, 0.7, 0.4, 0.4],
                   [0.5, 0.5, 0.3, 0.3],
                   [0.6, 0.3, 0.2, 0.2],
                   [0.3, 0.3, 0.2, 0.2]
                ]).unsqueeze(0) # for batch




In [45]:
iouloss = IoULoss(method = 'IoU')

In [46]:
iouloss(pred, gt)

tensor([[[8.2014e-01],
         [3.5763e-07],
         [1.0000e+00],
         [1.0000e+00]]])

In [47]:
0.0625 / (0.25 + 0.16 - 0.0625)

0.17985611510791366

In [48]:
3.5763e-07

3.5763e-07